In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define PointNet architecture
def PointNet(num_classes):
    input_points = layers.Input(shape=(None, 3))  # Input: point cloud of shape (num_points, 3)

    # MLP to process point cloud
    x = layers.Conv1D(64, 1, activation='relu')(input_points)
    x = layers.Conv1D(128, 1, activation='relu')(x)
    x = layers.Conv1D(1024, 1, activation='relu')(x)

    # Max pooling to get global feature
    x = layers.GlobalMaxPooling1D()(x)

    # Fully connected layers
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dense(256, activation='relu')(x)
    output = layers.Dense(num_classes, activation='softmax')(x)  # Output: class scores

    # Build and compile the model
    model = models.Model(inputs=input_points, outputs=output)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Load the model
model = PointNet(num_classes=40)  # ModelNet40 has 40 classes

2024-10-02 09:39:21.248030: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-02 09:39:21.280519: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-02 09:39:21.280543: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-02 09:39:21.281468: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-02 09:39:21.285851: I tensorflow/core/platform/cpu_feature_guar

In [ ]:
import trimesh
import numpy as np
import os

# Function to load and preprocess a single .off file as a point cloud
def load_off_file(off_filename, num_points=1024):
    # Load the mesh using trimesh
    mesh = trimesh.load_mesh(off_filename)
    
    # Sample points from the mesh
    point_cloud = mesh.sample(num_points)
    
    # Normalize the point cloud (optional, but typically done)
    point_cloud -= np.mean(point_cloud, axis=0)  # Centering
    point_cloud /= np.max(np.linalg.norm(point_cloud, axis=1))  # Scaling
    
    return point_cloud

# Function to load all .off files in a directory and their labels
def load_off_dataset(directory, num_points=1024):
    point_clouds = []
    labels = []
    
    # Walk through the directory to find all .off files
    for subdir, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.off'):
                # Full path to the file
                file_path = os.path.join(subdir, file)
                
                # Extract label from the subdirectory (e.g., 'bottle' or 'chair')
                label = os.path.basename(subdir)
                
                # Load the point cloud from the .off file
                point_cloud = load_off_file(file_path, num_points=num_points)
                
                # Append the point cloud and label
                point_clouds.append(point_cloud)
                labels.append(label)
    
    # Convert to numpy arrays
    point_clouds = np.array(point_clouds)
    labels = np.array(labels)
    
    return point_clouds, labels

# Example of loading the dataset
dataset_dir = 'archive/ModelNet40/'
train_data, train_labels = load_off_dataset(dataset_dir)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x712e2c7a7640>>
Traceback (most recent call last):
  File "/home/andreas/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x712e2c7a7640>>
Traceback (most recent call last):
  File "/home/andreas/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x712e2c7a7640>>
Traceback (most recent call last):
  File "/home/andreas/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_

In [9]:
# Preprocess: Randomly shuffle the data
indices = np.arange(len(train_labels))
np.random.shuffle(indices)
train_data = train_data[indices]
train_labels = train_labels[indices]

# Check the shapes of the data
print("Train data shape:", train_data.shape)  # Should be (num_samples, 1024, 3)
print("Train labels shape:", train_labels.shape)

Train data shape: (12311, 1024, 3)
Train labels shape: (12311,)


In [14]:
LABEL_MAP = {
    'airplane': 0,
    'bathtub': 1,
    'bed': 2,
    'bench': 3,
    'bookshelf': 4,
    'bottle': 5,
    'bowl': 6,
    'car': 7,
    'chair': 8,
    'cone': 9,
    'cup': 10,
    'curtain': 11,
    'desk': 12,
    'door': 13,
    'dresser': 14,
    'flower_pot': 15,
    'glass_box': 16,
    'guitar': 17,
    'keyboard': 18,
    'lamp': 19,
    'laptop': 20,
    'mantel': 21,
    'monitor': 22,
    'night_stand': 23,
    'person': 24,
    'piano': 25,
    'plant': 26,
    'radio': 27,
    'range_hood': 28,
    'sink': 29,
    'sofa': 30,
    'stairs': 31,
    'stool': 32,
    'table': 33,
    'tent': 34,
    'toilet': 35,
    'tv_stand': 36,
    'vase': 37,
    'wardrobe': 38,
    'xbox': 39
}

In [13]:
import trimesh
import numpy as np
import os

# Function to load and preprocess a single .off file as a point cloud
def load_off_file(off_filename, num_points=1024):
    # Load the mesh using trimesh
    mesh = trimesh.load_mesh(off_filename)
    
    # Sample points from the mesh
    point_cloud = mesh.sample(num_points)
    
    # Normalize the point cloud (optional, but typically done)
    point_cloud -= np.mean(point_cloud, axis=0)  # Centering
    point_cloud /= np.max(np.linalg.norm(point_cloud, axis=1))  # Scaling
    
    return point_cloud

# Function to load all .off files in a directory and their labels
def load_off_dataset(directory, num_points=1024):
    point_clouds = []
    labels = []
    
    # Walk through the directory to find all .off files
    for subdir, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.off'):
                # Full path to the file
                file_path = os.path.join(subdir, file)
                
                # Extract label from the subdirectory (e.g., 'bottle' or 'chair')
                label_str = os.path.basename(subdir)
                
                # Convert label string to a numerical label using the mapping
                label = LABEL_MAP[label_str]
                
                # Load the point cloud from the .off file
                point_cloud = load_off_file(file_path, num_points=num_points)
                
                # Append the point cloud and label
                point_clouds.append(point_cloud)
                labels.append(label)
    
    # Convert to numpy arrays
    point_clouds = np.array(point_clouds)
    labels = np.array(labels)
    
    return point_clouds, labels

# Example of loading the dataset
dataset_dir = 'archive/ModelNet40/'
train_data, train_labels = load_off_dataset(dataset_dir)

# Shuffle the data
indices = np.arange(len(train_labels))
np.random.shuffle(indices)
train_data = train_data[indices]
train_labels = train_labels[indices]

# Check the shapes of the data
print("Train data shape:", train_data.shape)  # Should be (num_samples, 1024, 3)
print("Train labels shape:", train_labels.shape)  # Should be (num_samples,)
print(train_labels)  # Should output integer labels

KeyError: 'train'

In [2]:
import trimesh
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder

# Function to load and preprocess a single .off file as a point cloud
def load_off_file(off_filename, num_points=1024):
    # Load the mesh using trimesh
    mesh = trimesh.load_mesh(off_filename)
    
    # Sample points from the mesh
    point_cloud = mesh.sample(num_points)
    
    # Normalize the point cloud (optional, but typically done)
    point_cloud -= np.mean(point_cloud, axis=0)  # Centering
    point_cloud /= np.max(np.linalg.norm(point_cloud, axis=1))  # Scaling
    
    return point_cloud

# Function to load all .off files in a directory and their labels
def load_off_dataset(directory, num_points=1024):
    point_clouds = []
    labels = []
    
    # Walk through the directory to find all .off files
    for subdir, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.off'):
                # Full path to the file
                file_path = os.path.join(subdir, file)
                
                # Extract label from the subdirectory (e.g., 'bottle' or 'chair')
                label = os.path.basename(subdir)
                
                # Load the point cloud from the .off file
                point_cloud = load_off_file(file_path, num_points=num_points)
                
                # Append the point cloud and label
                point_clouds.append(point_cloud)
                labels.append(label)
    
    # Convert to numpy arrays
    point_clouds = np.array(point_clouds)
    labels = np.array(labels)
    
    return point_clouds, labels

# Load the dataset
dataset_dir = 'archive/ModelNet40/'
train_data, train_labels = load_off_dataset(dataset_dir)

# Use LabelEncoder to convert string labels to numeric labels
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)  # Converts string labels to integers

# Shuffle the data
indices = np.arange(len(train_labels))
np.random.shuffle(indices)
train_data = train_data[indices]
train_labels = train_labels[indices]

# Check the shapes of the data
print("Train data shape:", train_data.shape)  # Should be (num_samples, 1024, 3)
print("Train labels shape:", train_labels.shape)  # Should be (num_samples,)
print("Example labels:", train_labels[:10])  # Should print the first 10 numeric labels

/home/andreas/.local/lib/python3.10/site-packages/trimesh/grouping.py:99: RuntimeWarning: invalid value encountered in cast
  stacked = np.column_stack(stacked).round().astype(np.int64)


Train data shape: (12311, 1024, 3)
Train labels shape: (12311,)
Example labels: [1 0 1 1 1 0 0 1 1 1]


In [11]:
print("Example labels:", train_labels)  

Example labels: [22 37 35 ... 14 23 12]


In [7]:
# Split the training data into "train" and "test" sets for evaluation
split_ratio = 0.8  # Use 80% for training and 20% for testing
split_index = int(len(train_data) * split_ratio)

# Train data
train_data_split = train_data[:split_index]
train_labels_split = train_labels[:split_index]

# Test data
test_data = train_data[split_index:]
test_labels = train_labels[split_index:]

# Predict class indices using the model on the test data
predicted_labels = model.predict(test_data)
predicted_class_indices = np.argmax(predicted_labels, axis=-1)

# Convert numeric labels back to string labels
predicted_class_names = label_encoder.inverse_transform(predicted_class_indices)
print(predicted_class_names[:20])  # Will print the original labels like 'bottle', 'chair'


77/77 [==============================] - 3s 42ms/step
['train' 'train' 'train' 'train' 'train' 'train' 'train' 'train' 'train'
 'train' 'train' 'train' 'train' 'train' 'train' 'train' 'train' 'train'
 'train' 'train']


In [8]:
import trimesh
import numpy as np
import os

# Function to load and preprocess a single .off file as a point cloud
def load_off_file(off_filename, num_points=1024):
    # Load the mesh using trimesh
    mesh = trimesh.load_mesh(off_filename)
    
    # Sample points from the mesh
    point_cloud = mesh.sample(num_points)
    
    # Normalize the point cloud (optional, but typically done)
    point_cloud -= np.mean(point_cloud, axis=0)  # Centering
    point_cloud /= np.max(np.linalg.norm(point_cloud, axis=1))  # Scaling
    
    return point_cloud

# Function to load all .off files in a directory and their labels
def load_off_dataset(directory, num_points=1024):
    point_clouds = []
    labels = []
    
    # Walk through the directory to find all .off files
    for subdir, dirs, files in os.walk(directory):
        # Skip the 'train' folder and process only class subdirectories
        if os.path.basename(subdir) == 'train':
            continue
        
        for file in files:
            if file.endswith('.off'):
                # Full path to the file
                file_path = os.path.join(subdir, file)
                
                # Extract the class label from the parent directory
                label = os.path.basename(os.path.dirname(subdir))
                
                # Load the point cloud from the .off file
                point_cloud = load_off_file(file_path, num_points=num_points)
                
                # Append the point cloud and label
                point_clouds.append(point_cloud)
                labels.append(label)
    
    # Convert to numpy arrays
    point_clouds = np.array(point_clouds)
    labels = np.array(labels)
    
    return point_clouds, labels

# Load the dataset
dataset_dir = 'archive/ModelNet40/'
train_data, train_labels = load_off_dataset(dataset_dir)

# Use LabelEncoder to convert string labels to numeric labels
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)  # Converts string labels to integers

# Shuffle the data
indices = np.arange(len(train_labels))
np.random.shuffle(indices)
train_data = train_data[indices]
train_labels = train_labels[indices]

# Check the shapes of the data
print("Train data shape:", train_data.shape)  # Should be (num_samples, 1024, 3)
print("Train labels shape:", train_labels.shape)  # Should be (num_samples,)
print("Example labels:", train_labels[:10])  # Should print the first 10 numeric labels

Train data shape: (2468, 1024, 3)
Train labels shape: (2468,)
Example labels: [22 37 35 34  6 22 37 26 25 14]


In [9]:
model.fit(train_data, train_labels, epochs=5, batch_size=32, validation_split=0.1)

Epoch 1/5
70/70 [==============================] - 14s 200ms/step - loss: 4.2652 - accuracy: 0.0873 - val_loss: 2.7393 - val_accuracy: 0.2470
Epoch 2/5
70/70 [==============================] - 14s 198ms/step - loss: 2.4374 - accuracy: 0.3003 - val_loss: 2.0571 - val_accuracy: 0.4130
Epoch 3/5
70/70 [==============================] - 14s 202ms/step - loss: 1.9761 - accuracy: 0.4183 - val_loss: 1.6988 - val_accuracy: 0.4413
Epoch 4/5
70/70 [==============================] - 14s 201ms/step - loss: 1.7061 - accuracy: 0.5061 - val_loss: 1.4549 - val_accuracy: 0.6073
Epoch 5/5
70/70 [==============================] - 14s 201ms/step - loss: 1.4962 - accuracy: 0.5628 - val_loss: 1.2700 - val_accuracy: 0.6437


In [10]:
# Predict class indices using the model
predicted_labels = model.predict(test_data)
predicted_class_indices = np.argmax(predicted_labels, axis=-1)

# Convert numeric labels back to string labels
predicted_class_names = label_encoder.inverse_transform(predicted_class_indices)
print(predicted_class_names)  # Will print the original labels like 'bottle', 'chair'


77/77 [==============================] - 3s 42ms/step
['bed' 'sofa' 'chair' ... 'bookshelf' 'dresser' 'bookshelf']


In [13]:
import open3d as o3d
import numpy as np

# Load a random point cloud from a .ply file
random_cloud = o3d.io.read_point_cloud("filtered.pcd")

# Convert the point cloud to a numpy array
random_points = np.asarray(random_cloud.points)

# Check the number of points and downsample if necessary
if random_points.shape[0] > 1024:
    random_points = random_points[:1024, :]  # Select the first 1024 points

# Normalize the point cloud (optional, depending on your preprocessing steps)
random_points = random_points - np.mean(random_points, axis=0)  # Centering
random_points = random_points / np.max(np.linalg.norm(random_points, axis=1))  # Scaling

# Ensure the shape is (num_points, 3)
print(random_points.shape)  # Should be (1024, 3)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
(1024, 3)


In [14]:
# Add a batch dimension to the point cloud (model expects (batch_size, num_points, 3))
random_points = np.expand_dims(random_points, axis=0)

# Predict the label of each point (if it's segmentation) or the class (if it's classification)
predicted_labels = model.predict(random_points)

# If it's classification, the output will be a single label
predicted_class = np.argmax(predicted_labels, axis=-1)
print(f"Predicted class for the random point cloud: {predicted_class}")

1/1 [==============================] - 0s 16ms/step
Predicted class for the random point cloud: [8]


In [19]:
# Print the shape of predicted_labels to confirm it's a batch of labels
print(f"Shape of predicted_labels: {predicted_labels.shape}")

# Assuming predicted_labels is an array of integers (class indices)
max_label = np.max(predicted_labels)  # Get the maximum label to normalize the colormap

# Ensure predicted_labels is an array
if isinstance(predicted_labels, np.ndarray):
    # Get a colormap and apply it to the labels
    cmap = plt.get_cmap("tab20")  # Colormap with 20 discrete colors

    # Apply the colormap to each predicted label
    # For each predicted label, cmap returns an RGBA tuple, we convert that to a numpy array
    colors = np.array([cmap(label / max_label) for label in predicted_labels])  # Normalize and get RGBA colors

    # colors will have shape (num_points, 4) (RGBA), we only need the RGB part
    colors = colors[:, :3]  # Extract the RGB channels (drop alpha)

    # Assuming random_cloud is the Open3D point cloud object
    random_cloud.colors = o3d.utility.Vector3dVector(colors)  # Apply the RGB colors

    # Visualize the point cloud with the predicted labels
    o3d.visualization.draw_geometries([random_cloud])
else:
    print("Error: predicted_labels is not an array.")

Shape of predicted_labels: ()
Error: predicted_labels is not an array.


In [20]:
# Assuming you are predicting on a point cloud
# Ensure that test_data has the right shape (batch_size, num_points, 3)
print(f"Shape of test_data: {test_data.shape}")  # Check the shape of the input data

# Ensure test_data has a batch dimension
if len(test_data.shape) == 2:  # If the shape is (num_points, 3)
    test_data = np.expand_dims(test_data, axis=0)  # Add batch dimension

# Now predict
predicted_labels = model.predict(test_data)
predicted_class_indices = np.argmax(predicted_labels, axis=-1)

# Now predicted_class_indices should be an array of labels
print(f"Shape of predicted_class_indices: {predicted_class_indices.shape}")

Shape of test_data: (2463, 1024, 3)
77/77 [==============================] - 3s 42ms/step
Shape of predicted_class_indices: (2463,)


In [21]:
# Ensure that test_data has the correct shape
if len(test_data.shape) == 2:
    test_data = np.expand_dims(test_data, axis=0)  # Add batch dimension

In [22]:
# Predict labels using the corrected test_data shape
predicted_labels = model.predict(test_data)
predicted_class_indices = np.argmax(predicted_labels, axis=-1)

# Check the shape of the predicted_class_indices
print(f"Shape of predicted_class_indices: {predicted_class_indices.shape}")

# Now apply the colormap as before
max_label = np.max(predicted_class_indices)

cmap = plt.get_cmap("tab20")
colors = np.array([cmap(label / max_label) for label in predicted_class_indices[0]])  # For the first point cloud in the batch

colors = colors[:, :3]  # Extract the RGB channels

random_cloud.colors = o3d.utility.Vector3dVector(colors)  # Apply the RGB colors

# Visualize the point cloud with the predicted labels
o3d.visualization.draw_geometries([random_cloud])

77/77 [==============================] - 3s 43ms/step
Shape of predicted_class_indices: (2463,)


TypeError: 'numpy.int64' object is not iterable

In [23]:
# Predict labels using the model
predicted_labels = model.predict(test_data)
predicted_class_indices = np.argmax(predicted_labels, axis=-1)

# Check the shape of the predicted labels to understand what the model is returning
print(f"Shape of predicted_class_indices: {predicted_class_indices.shape}")

# If the shape is correct, continue with colormap application
if len(predicted_class_indices.shape) == 2:  # (batch_size, num_points)
    # Now apply the colormap to the first point cloud in the batch
    max_label = np.max(predicted_class_indices)
    cmap = plt.get_cmap("tab20")
    
    # Apply the colormap to each label in the first point cloud
    colors = np.array([cmap(label / max_label) for label in predicted_class_indices[0]])  # First point cloud
    
    # colors will have shape (num_points, 4), we only need the RGB part
    colors = colors[:, :3]  # Extract RGB channels
    
    # Assuming random_cloud is the Open3D point cloud object
    random_cloud.colors = o3d.utility.Vector3dVector(colors)  # Apply the RGB colors
    
    # Visualize the point cloud with the predicted labels
    o3d.visualization.draw_geometries([random_cloud])
else:
    print("Error: predicted_class_indices has an unexpected shape. Please check your model output.")


77/77 [==============================] - 3s 42ms/step
Shape of predicted_class_indices: (2463,)
Error: predicted_class_indices has an unexpected shape. Please check your model output.
